In [26]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").getOrCreate()

In [27]:
transRDD = spark.sparkContext.textFile("trans240.csv")
custRDD =  spark.sparkContext.textFile("cust.csv")

In [28]:
#1. For each month, show the number of distinct players, and the total cost, the results are sorted by month

In [29]:
kq = transRDD.map(lambda n: n.split(",")).map(lambda n: (n[1].split("-")[0], float(n[3]))).reduceByKey(lambda n, m: n + m) \
.join(transRDD.map(lambda n: n.split(",")).map(lambda n: (n[1].split("-")[0], n[2])).distinct().groupByKey().mapValues(len))

In [30]:
def thang_so(month):
  months = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
  }
  return months[month]
temp = kq.map(lambda n: (thang_so(n[0]), (n[0], n[1][1], n[1][0])))

In [31]:
temp.sortByKey().map(lambda n: n[1]).collect()

[('Jan', 9, 2088.2099999999996),
 ('Feb', 9, 2261.77),
 ('Mar', 9, 1727.3200000000002),
 ('Apr', 9, 1223.06),
 ('May', 10, 2709.75),
 ('Jun', 9, 1891.62),
 ('Jul', 8, 2550.51),
 ('Aug', 9, 1498.7),
 ('Sep', 8, 1444.2800000000002),
 ('Oct', 10, 3064.28),
 ('Nov', 8, 1682.6899999999998),
 ('Dec', 9, 2510.7900000000004)]

In [7]:
#2. For each month, show the name three youngest player, the results are sorted by the month

In [32]:
kq1 = transRDD.map(lambda n: n.split(",")).map(lambda n: (n[2], n[1].split('-')[0])) \
.join(custRDD.map(lambda n: n.split(",")).map(lambda n: (n[0], (n[1], int(n[3]))))) 

In [33]:
temp1 = kq1.map(lambda n: n[1]).distinct().groupByKey()\
.map(lambda n: (n[0], sorted(list(n[1]), key = lambda m: m[1])[:3]))

In [34]:
temp1.map(lambda n: (thang_so(n[0]), n)).sortByKey().map(lambda n: n[1]).collect()

[('Jan', [('Sherri', 34), ('Malcolm', 39), ('Patrick', 42)]),
 ('Feb', [('Malcolm', 39), ('Patrick', 42), ('Elsie', 43)]),
 ('Mar', [('Sherri', 34), ('Malcolm', 39), ('Elsie', 43)]),
 ('Apr', [('Sherri', 34), ('Malcolm', 39), ('Elsie', 43)]),
 ('May', [('Sherri', 34), ('Malcolm', 39), ('Patrick', 42)]),
 ('Jun', [('Sherri', 34), ('Malcolm', 39), ('Elsie', 43)]),
 ('Jul', [('Sherri', 34), ('Patrick', 42), ('Elsie', 43)]),
 ('Aug', [('Malcolm', 39), ('Patrick', 42), ('Elsie', 43)]),
 ('Sep', [('Malcolm', 39), ('Elsie', 43), ('Kristina', 55)]),
 ('Oct', [('Sherri', 34), ('Malcolm', 39), ('Patrick', 42)]),
 ('Nov', [('Sherri', 34), ('Malcolm', 39), ('Kristina', 55)]),
 ('Dec', [('Sherri', 34), ('Malcolm', 39), ('Patrick', 42)])]

In [35]:
#3. For each state, show the number of distinct players of each state, the results are sorted by the number of players

In [36]:
transRDD.map(lambda n: n.split(",")).map(lambda n: (n[7], n[2])).distinct().groupByKey().mapValues(len).sortBy(lambda n: n[1]).collect()

[('Wisconsin', 1),
 ('Iowa', 1),
 ('Florida', 1),
 ('Utah', 1),
 ('New Jersey', 1),
 ('Louisiana', 1),
 ('Kentucky', 1),
 ('Oklahoma', 1),
 ('Hawaii', 2),
 ('Nebraska', 2),
 ('New York', 2),
 ('Minnesota', 2),
 ('Texas', 2),
 ('Arizona', 2),
 ('Georgia', 2),
 ('Colorado', 2),
 ('Virginia', 2),
 ('Massachusetts', 2),
 ('Tennessee', 3),
 ('Illinois', 3),
 ('South Carolina', 3),
 ('Ohio', 3),
 ('Nevada', 3),
 ('Pennsylvania', 3),
 ('Washington', 3),
 ('California', 6)]

In [37]:
#4. For each state, show the name of three oldest player in each state, the results are sorted by the state

In [38]:
kq2 = transRDD.map(lambda n: n.split(",")).map(lambda n: (n[2], n[7])) \
.join(custRDD.map(lambda n: n.split(",")).map(lambda n: (n[0], (n[1], int(n[3])))))

In [39]:
kq2.map(lambda n: n[1]).distinct().groupByKey()\
.map(lambda n: (n[0], sorted(list(n[1]), key=lambda m: m[1], reverse=True)[:3])).collect()

[('Louisiana', [('Karen', 74)]),
 ('New York', [('Karen', 74), ('Patrick', 42)]),
 ('Nevada', [('Karen', 74), ('Gretchen', 66), ('Elsie', 43)]),
 ('Virginia', [('Karen', 74), ('Elsie', 43)]),
 ('Wisconsin', [('Sherri', 34)]),
 ('Illinois', [('Gretchen', 66), ('Hazel', 63), ('Malcolm', 39)]),
 ('Colorado', [('Gretchen', 66), ('Hazel', 63)]),
 ('Tennessee', [('Paige', 74), ('Hazel', 63), ('Kristina', 55)]),
 ('Hawaii', [('Dolores', 60), ('Kristina', 55)]),
 ('Nebraska', [('Hazel', 63), ('Kristina', 55)]),
 ('Washington', [('Hazel', 63), ('Dolores', 60), ('Kristina', 55)]),
 ('Oklahoma', [('Paige', 74)]),
 ('New Jersey', [('Hazel', 63)]),
 ('Texas', [('Gretchen', 66), ('Sherri', 34)]),
 ('California', [('Paige', 74), ('Hazel', 63), ('Dolores', 60)]),
 ('South Carolina', [('Karen', 74), ('Gretchen', 66), ('Dolores', 60)]),
 ('Arizona', [('Kristina', 55), ('Malcolm', 39)]),
 ('Ohio', [('Kristina', 55), ('Patrick', 42), ('Malcolm', 39)]),
 ('Pennsylvania', [('Kristina', 55), ('Elsie', 43), (

In [40]:
#5. For each state, show the average age of players in each state, the results are sorted by the average age

In [41]:
kq3 = transRDD.map(lambda n: n.split(",")).map(lambda n: (n[2], n[7])) \
.join(custRDD.map(lambda n: n.split(",")).map(lambda n: (n[0], int(n[3]))))

In [42]:
kq3.map(lambda n: (n[1][0], (n[1][1], 1))).distinct() \
.reduceByKey(lambda n, m: (n[0] + m[0], n[1] + m[1])).map(lambda n: (n[0], n[1][0] / n[1][1])).sortBy(lambda n: n[1]).collect()

[('Wisconsin', 34.0),
 ('Minnesota', 41.0),
 ('Iowa', 42.0),
 ('Florida', 42.0),
 ('Ohio', 45.333333333333336),
 ('Pennsylvania', 46.666666666666664),
 ('Arizona', 47.0),
 ('Texas', 50.0),
 ('California', 52.166666666666664),
 ('Illinois', 56.0),
 ('Hawaii', 57.5),
 ('New York', 58.0),
 ('Virginia', 58.5),
 ('Nebraska', 59.0),
 ('Washington', 59.333333333333336),
 ('Nevada', 61.0),
 ('New Jersey', 63.0),
 ('Utah', 63.0),
 ('Kentucky', 63.0),
 ('Tennessee', 64.0),
 ('Colorado', 64.5),
 ('South Carolina', 66.66666666666667),
 ('Massachusetts', 67.0),
 ('Georgia', 68.5),
 ('Oklahoma', 74.0),
 ('Louisiana', 74.0)]